# Dense Opical Flow analysis

By Marc van  Zyl   


This notebook uses the OpenCV dense optical flow algorithm by Gunnar Farneback to measure distances to target objects.

Optical flow is calculated over the entire image and the results are extracted in the vicinity of the centers of the Aruco target markers (these are found through a separate process)


In [ ]:
import numpy as np
import cv2, os
from cv2 import aruco
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
%matplotlib inline

## Aruco configuration
The following line of the board was updated to reflect the correct scale of the board.  This is necessary becuase the `board` object is used later   
`board = aruco.CharucoBoard_create(7, 5, .04026, .8*.04026, aruco_dict)`


In [ ]:
board_size = 'experimental setup'
if board_size == '7x5':
    chessboard_num_squares_across = 7
    chessboard_num_squares_up = 5
    chessboard_square_size = 0.04026
    chessboard_aruco_ratio = 0.8   # this is a fraction of chessboard_square_size
    aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_250)


if board_size == '12x8':
    chessboard_num_squares_across = 12
    chessboard_num_squares_up = 8
    chessboard_square_size = 1
    chessboard_aruco_ratio = 0.7   # this is a fraction of chessboard_square_size
    aruco_dict = aruco.Dictionary_get(aruco.DICT_5X5_250)

elif board_size == 'experimental setup':   
    aruco_dict = aruco.Dictionary_get(aruco.DICT_4X4_50)





Load the camera calibration matrices

In [ ]:
import pickle

cam_calOF = pickle.load( open('OFCameraCalibration.p', 'rb'))

In [ ]:
cam_calOF.keys()

## Camera properties

## Camera features:
- sensor size = 3.68 x 2.76 mm  
- sensor resolution  = 3280 × 2464
- focal length = 3.04 mm

$$ d_{mm} = \frac{pix \times 3.68}{3280} $$

The depth can now be found
$$ Z = \frac{T \times f}{d_{mm}} $$

In [ ]:
import pandas as pd

Function to increase the contrast in the picture to make the marker identification easier

In [ ]:
# adapted from here https://stackoverflow.com/questions/39308030/how-do-i-increase-the-contrast-of-an-image-in-python-opencv
def increase_contrast(img, clipLimit=3.0, verbose=False):

    if clipLimit>0.0:
        #-----Converting image to LAB Color model----------------------------------- 
        lab= cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        if verbose:
            cv2.imshow("lab",lab)

        #-----Splitting the LAB image to different channels-------------------------
        l, a, b = cv2.split(lab)
        if verbose:
            cv2.imshow('l_channel', l)
            cv2.imshow('a_channel', a)
            cv2.imshow('b_channel', b)

        #-----Applying CLAHE to L-channel-------------------------------------------
        clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=(8,8))
        cl = clahe.apply(l)
        if verbose:
            cv2.imshow('CLAHE output', cl)

        #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
        limg = cv2.merge((cl,a,b))
        if verbose:
            cv2.imshow('limg', limg)

        #-----Converting image from LAB Color model to RGB model--------------------
        final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
        if verbose:
            cv2.imshow('final', final)
    else:
        final = img.copy()

    return final

## Function to find the centers of the Aruco targets
This function:
1. finds the locations of the corners of the aruco squares (`cv2.aruco.detectMarkers`)
1. if markers were found interpolates to find the checkerboard markers between them (`cv2.aruco.interpolateCornersCharuco`)
1. zooms into each checkerboard corner to get sub-pixel accuracy using (`cv2.cornerSubPix`)

This function matches the corners found in the Left and the Right cameras for comparison

In [ ]:
def find_common_corners(cornersL, idsL, cornersR, idsR, verbose=False):
    # find the list of corners common to both images
    # get a list of the ids
    listIdsL = idsL.reshape((1,-1))[0].tolist()
    listIdsR = idsR.reshape((1,-1))[0].tolist()
    
    # this selects only the points that are in listIdsA and listIdsC
    commonIds = list(set(listIdsL).intersection(set(listIdsR)))

    if verbose:
        print('Found {} common IDS between L:({}) and R:({})'.format(len(commonIds),
                                                                     len(idsL),
                                                                     len(idsR)))

    # Reindex L camera data by taking out only the common points 
    re_index = [ listIdsL.index(ind) for ind in commonIds]
    idsL_new = idsL[re_index]
    cornersL_new = np.array(cornersL)[re_index]
    cornersL_new = [x for x in cornersL_new]

    # Reindex R camera data by taking out only the common points
    re_index = [ listIdsR.index(ind) for ind in commonIds]
    idsR_new = idsR[re_index]
    cornersR_new = np.array(cornersR)[re_index]
    cornersR_new = [x for x in cornersR_new]

    return np.array(cornersL_new), idsL_new, np.array(cornersR_new), idsR_new

In [ ]:
def find_charuco_marker_corners(img, aruco_dict, clipLimit=3.0, verbose=False):

    # These are parameters used by the cv2.cornerSubPix function
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10000, 1e-11)

    # increase the contraxt
    img = increase_contrast(img, clipLimit=clipLimit)


 
    # convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    

    # find the aruco corners and the ids of each corner
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(gray, aruco_dict)

    if verbose:
        print('Found {} aruco marker corners'.format(len(ids)))
        
    
    #  now find the central points of the aruco markers by averaging the four corner points
    if  len(ids) > 0:
        # SUB PIXEL DETECTION
        for corner in corners:
            if verbose:
                print('Sub pixel optimization:')
                print(corner)
            cv2.cornerSubPix(gray, corner,
                             winSize = (4,4),
                             zeroZone = (-1,-1),
                             criteria = criteria)
            if verbose:
                print(corner)
                print('+++')




    return np.array(corners), ids, gray.shape

        

In [ ]:
def find_aruco_center(corners, ids):
    # find the center point of the 4 corner of the aruco markers
    centers = []
    for cnrs in corners:
        center = np.array((np.average(cnrs[0][:,0]), np.average(cnrs[0][:,1])))
        centers.append(center)
    centers = np.array(centers)
    centers = np.array(centers).reshape((-1,1,2))
    center_ids = np.arange(centers.shape[0]).reshape(-1,1)
    
    return centers, ids

In [ ]:
image_number = 0

# Load the list of files

In [ ]:
datadir = "/Users/marcvanzyl/Google Drive/ScienceFair/data/Dense Lot/Speed_075/"


video_files = np.array([f for f in os.listdir(datadir) if f.endswith(".h264") ])

# just sorts the files according to the number so we match picture 1_A with 1_C etc
orderR = np.argsort([int((p.split('_')[-1]).split('.')[0]) for p in video_files])
video_files = video_files[orderR]


In [ ]:
video_files

# Run the analysis

In [ ]:
# set verbose = True to make plots of each frame
verbose =  True
def analyze_files(video_files):
    
    for file in video_files:

        result_list = []

        counter = 0

        # the size of the square to average the flow over
        center_range = 10

        cap = cv2.VideoCapture(datadir + file)
        print("Working with: {}".format(file))

        ret, frame1 = cap.read()
        last_pic_3_color = frame1 #cv2.undistort(frame1,cam_calOF['mtx'],cam_calOF['dist'],None)
        last_pic_3_grey = cv2.cvtColor(last_pic_3_color,cv2.COLOR_BGR2GRAY)


        ret, frame1 = cap.read()
        last_pic_2_color = frame1 #cv2.undistort(frame1,cam_calOF['mtx'],cam_calOF['dist'],None)
        last_pic_2_grey = cv2.cvtColor(last_pic_2_color,cv2.COLOR_BGR2GRAY)


        ret, frame1 = cap.read()
        last_pic_1_color = frame1 #cv2.undistort(frame1,cam_calOF['mtx'],cam_calOF['dist'],None)
        last_pic_1_grey = cv2.cvtColor(last_pic_1_color,cv2.COLOR_BGR2GRAY)


        hsv = np.zeros_like(frame1)
        hsv[...,1] = 255
        flow = np.zeros_like(frame1)

        # this creates the array for the sorted centers 
        sorted_centers = np.zeros([10,2])

        while(cap.isOpened()):
            ret, curr_pic_color = cap.read()
            # ret will be False if there are no more frames
            if ret == False:
                break

            counter += 1
            if counter > 120:
                break
            curr_pic_color = curr_pic_color #cv2.undistort(curr_pic_color,cam_calOF['mtx'],cam_calOF['dist'],None)

            curr_pic_grey = cv2.cvtColor(curr_pic_color, cv2.COLOR_BGR2GRAY)

            if counter > 1:
                flags = cv2.OPTFLOW_USE_INITIAL_FLOW
            else:
                flags = 0

            flow = cv2.calcOpticalFlowFarneback(last_pic_3_grey, curr_pic_grey, flow, 0.5, 3, 50, 6, 7, 1.2, flags)

            # For printing
            # mag = simple_norm(np.power(flow[...,0], 3), 3, 25, 0, 255)
            #hsv[...,0] = 0
            #hsv[...,2] = mag
            #rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
            #cv2.imshow('curr_pic_color',rgb)

            print('Image Frame Number: {}'.format(counter))


            # find the four corners of each aruco marker
            arucoCorners, arucoIds, imsize =  find_charuco_marker_corners(curr_pic_color, 
                                                                aruco_dict, clipLimit=2.0, 
                                                                verbose=False)

            # now find the center point of each aruco marker 
            centers, center_ids = find_aruco_center(arucoCorners, arucoIds)

            # next sort the centers by center_ids from 0 to 9
            center_ids = center_ids.reshape(-1)
            centers = centers.reshape(-1,2)
            print(center_ids)

            # insert the centers into the sorted centers array in the right order
            for i, ids in enumerate(center_ids):
                if (ids < 10):
                    sorted_centers[ids] = centers[i]

            frame_result_list = []

            # for each center extract the 20x20 pixels around it and find the mean
            # of the opptical flow in the x and y directions
            for center in sorted_centers:
                centerx = int(center[1])
                centery = int(center[0])
                point_result_list = [flow[centerx-center_range:centerx+center_range,
                                          centery-center_range:centery+center_range][:,:,0].mean(),
                                     flow[centerx-center_range:centerx+center_range,
                                          centery-center_range:centery+center_range][:,:,1].mean()]
                frame_result_list.append(point_result_list)

                print("x: {}  Y: {}".format(point_result_list[0],point_result_list[1]))

            result_list.append(frame_result_list)

            if verbose:
                x = sorted_centers[:,0]
                y = sorted_centers[:,1]
                mag = (cv2.multiply(cv2.add(flow[...,0],-.20),30))

                mag[mag<0.0] = 0
                mag[mag>255] = 255

                print("mean: {} max: {}  min: {} {}".format(mag.mean(), mag.max(), mag.min(), flow.shape))

                # edge detection for the outlines
                edges = cv2.Canny(curr_pic_color, 50, 100)


                hsv[...,0] = 0
                #hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
                hsv[...,2] = mag
                rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)

                #mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
                #hsv[..., 0] = ang * 180 / np.pi / 2
                #hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
                #rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)


                annot_font = {'fontname':'Arial', 'size':'14','weight':'bold'}

                fig, axs = plt.subplots(figsize=(36,24))
                rgb[edges>100,1] = 255 
                axs.imshow(rgb)
                axs.scatter(x,y, color='r')
                for i in range(10):
                    axs.annotate('{}'.format(int(i)), (x[i]+20, y[i]+30), color='r',  **annot_font)


                plt.show()

            last_pic_3_grey = np.copy(last_pic_2_grey)

            last_pic_2_grey = np.copy(last_pic_1_grey)

            last_pic_1_grey = np.copy(curr_pic_grey)

        pickle.dump(np.array(result_list)[5:105], open('{}{}_res_3.p'.format(datadir, file[:-5]), 'wb'))
        result_list = []
        cap.release()
        cap = None

In [ ]:
analyze_files(video_files)

In [ ]:
speeds = ['075', '015', '030', '045','060']
for speed in speeds:
    print(speed)
    
    datadir = "/Users/marcvanzyl/Google Drive/ScienceFair/data/Dense Lot/Speed_{}/".format(speed)
    print(datadir)

    video_files = np.array([f for f in os.listdir(datadir) if f.endswith(".h264") ])
    print(video_files)
    # just sorts the files according to the number so we match picture 1_A with 1_C etc
    orderR = np.argsort([int((p.split('_')[-1]).split('.')[0]) for p in video_files])
    video_files = video_files[orderR]
    
    analyze_files(video_files)

In [ ]:
# save the results for analysis
pickle.dump(np.array(result_list), open('{}{}_res.p'.format(datadir, file[:-5]), 'wb'))

In [ ]:
'{}{}_res.p'.format(datadir, file[:-5])

In [ ]:
np.array(result_list)[5:105].shape